# PART 1 - Get the data and prepare it

In [2]:
import pandas as pd

In [23]:
# table with postal codes of canada
url = "https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&direction=prev&oldid=926287641"
# get all tables from the site
dfs = pd.read_html(url)
# get the borough data
df = dfs[0]

**preparing the data**

In [24]:
# drop rows without a borough
df.drop(df[df.Borough == 'Not assigned'].index, inplace=True)

In [26]:
# if the neighbourhood is not assigned, use the borough name instead
df.loc[df["Neighbourhood"] == 'Not assigned', "Neighbourhood"] = df["Borough"]

In [37]:
# Group the data by postcode and borough
grouped_df = df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()

In [40]:
grouped_df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [36]:
# print the shape of the dataframe
grouped_df.shape

(103, 3)

# PART 2 - Add geospatial data